In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [4]:
# Dados Entrada

ger = ['Produtor A', 'Produtor B']
Ng = len(ger)
pmax_ter = [100, 80] # MW
c_ener_ger = [12, 20]

loads = ['Consumidor 1', 'Consumidor 2']
Nc = len(loads)
demanda_energia = [100, 50]
c_ener_load = [40, 35]

In [5]:
# Criar Modelo Concreto
model = ConcreteModel()

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener_ger = Param(model.ger, initialize={ger[i]: c_ener_ger[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: pmax_ter[i] for i in range(Ng)})

#Cargas:
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: demanda_energia[i] for i in range(Nc)})
model.c_ener_load = Param(model.loads, initialize={loads[i]: c_ener_load[i] for i in range(Nc)})

In [6]:
#Variáveis:
model.p_ger = Var(model.ger, domain=NonNegativeReals)
model.p_load = Var(model.loads, domain=NonNegativeReals)

In [7]:
#Restrições
def balanco(model):
    energia_vendida = sum(model.p_ger[gerador] for gerador in model.ger)
    energia_comprada = sum(model.p_load[carga] for carga in model.loads)
    return energia_vendida == energia_comprada
model.balanco = Constraint(rule = balanco)

def limite_gerador_superior(model, gerador):
    return model.p_ger[gerador] <= model.pmax_ter[gerador]
model.limite_gerador = Constraint(model.ger, rule=limite_gerador_superior)

def limite_carga_superior(model, carga):
    return model.p_load[carga] <= model.dem[carga]
model.limite_carga = Constraint(model.loads, rule=limite_carga_superior)

def limite_gerador_inferior(model, gerador):
    return model.p_ger[gerador] >= 0
model.limite_gerador_inf = Constraint(model.ger, rule=limite_gerador_inferior)

def limite_carga_inferior(model, carga):
    return model.p_load[carga] >= 0
model.limite_carga_inf = Constraint(model.loads, rule=limite_carga_inferior)

In [8]:
#Função Objetivo:
def objective_rule(model):
    compra_total = sum(model.p_load[carga] * model.c_ener_load[carga] for carga in model.loads)
    venda_total = sum(model.p_ger[gerador] * model.c_ener_ger[gerador] for gerador in model.ger)
    return compra_total - venda_total
model.obj = Objective(rule=objective_rule, sense=maximize)

In [9]:
solver = SolverFactory('glpk')
# solver = SolverFactory('ipopt',executable='/content/ipopt')

resultados = solver.solve(model, tee=False)

In [10]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.obj), '\n')

#Energias Vendidas:
print('Energias Vendidas:')
for gerador in model.ger:
    print(f'{gerador}:', value(model.p_ger[gerador]))
    print()

#Energias Compradas:
print('Energias Compradas:')
for carga in model.loads:
    print(f'{carga}:', value(model.p_load[carga]))
    print()

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 3550.0 

Energias Vendidas:
Produtor A: 100.0

Produtor B: 50.0

Energias Compradas:
Consumidor 1: 100.0

Consumidor 2: 50.0

